In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
import scipy.fftpack
%matplotlib inline

In [ ]:
from analysis.lib.nv import detection
folder= tb.latest_data('Entangle')
# folder = "D:/measuring/data/20170308/105316_PQ_single_click_ent_111no2_SIL2_phase_stab"
a = m2.M2Analysis(folder)

# stitch the different part and files together, make and save a plot of the data.
s = []
u_1 = []
v_1 = []
u_2 = []
v_2 = []
w = []

pid_counts_1 = a.g['adwindata']['pid_counts_1'].value
pid_counts_2 = a.g['adwindata']['pid_counts_2'].value
max_repetitions = a.g['adwindata']['completed_reps'].value
g_0 = a.g.attrs['Phase_Msmt_g_0']
visibility = a.g.attrs['Phase_Msmt_Vis']

delay = a.g.attrs['count_int_time']/1000.0
pid_cycles = a.g.attrs['pid_points']

# stitch
for i in xrange(max_repetitions):
    for j in xrange(pid_cycles):
        if (pid_counts_1[j+i*pid_cycles] != -1):
            u_1.append(pid_counts_1[j+i*pid_cycles])
            u_2.append(pid_counts_2[j+i*pid_cycles])
            cosval = 2*(float(u_1[-1])/(float(u_1[-1])+float(u_2[-1])*g_0)-0.5)*visibility
            cosval = cosval if np.abs(cosval) < 1 else (1.0 * np.sign(cosval))
            angle1 = np.arccos(cosval)
                
            v_1.append(180.0*angle1/(np.pi))
#         w.append(pid_counts_1[j+i*pid_cycles]*1.04/(pid_counts_1[j+i*pid_cycles]*1.04+pid_counts_2[j+i*pid_cycles]*1.0))

t = np.arange(0, (len(u_1)*delay), (delay))

# counts
fig = plt.figure(figsize=(17,6))
ax = plt.subplot(211)
# plt.plot(t, s, 'y')
plt.plot(t, u_1, 'r')
plt.title('Counts ZPL detector 1 {0}'.format(folder))
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('counts')
ax2 = plt.subplot(212)
plt.plot(t, u_2, 'r')
plt.title('Counts ZPL detector 2 {0}'.format(folder))
ax2.set_xlabel('elapsed time (milliseconds)')
ax2.set_ylabel('counts')
# ax3 = plt.subplot(313)
# plt.plot(t, w, 'r')
# plt.title('error signal {0}'.format(folder))
# ax3.set_xlabel('elapsed time (milliseconds)')
# ax3.set_ylabel('error signal')
plt.tight_layout()
# plt.show()

fig.savefig(os.path.join(folder, 'trace_counts.png'))

zoom_1 = 5*(pid_cycles)
zoom_2 = 6*(pid_cycles)

# Zoomed counts
fig = plt.figure(figsize=(17,6))
ax = plt.subplot(211)
# plt.plot(t[0:zoom], s[0:zoom], 'y')
plt.plot(t[zoom_1:zoom_2], u_1[zoom_1:zoom_2], 'r')

plt.title('Counts ZPL detector 1'.format(folder))
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('counts')

ax2 = plt.subplot(212)
plt.plot(t[zoom_1:zoom_2], u_2[zoom_1:zoom_2], 'r')

plt.title('Counts ZPL detector 2'.format(folder))
ax2.set_xlabel('elapsed time (milliseconds)')
ax2.set_ylabel('counts')
plt.tight_layout()

fig.savefig(os.path.join(folder, 'trace_counts_zoomed.png'))

# Angle
fig = plt.figure(figsize=(17,6))
ax = plt.subplot(111)
plt.plot(t, v_1, 'r')
plt.title('Phase of ZPL photons {0}'.format(folder))
plt.ylim([0,180])
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('angle ($^o$)')

fig.savefig(os.path.join(folder, 'trace_angle.png'))

# std dev
fig = plt.figure()
ax = plt.subplot(111)
plt.plot(np.sqrt(np.var(np.reshape(v_1,[pid_cycles,-1]),axis=1)))
plt.title('Standard deviation of Phase {0}'.format(folder))
ax.set_xlabel('pid cycles (-)')
ax.set_ylabel('std dev ($^o$)')

In [ ]:
# fourier transform of the sample part

pid_counts_1 = a.g['adwindata']['pid_counts_1'].value
sample_counts_1 = a.g['adwindata']['sampling_counts_1'].value
max_repetitions = a.g['adwindata']['completed_reps'].value

delay = a.g.attrs['count_int_time']/1000.0
pid_cycles = a.g.attrs['pid_points']
sample_cycles = a.g.attrs['sample_points']

# stitch
v_1 = []
for i in xrange(max_repetitions):
    for j in xrange(pid_cycles):
         v_1.append(pid_counts_1[j+i*pid_cycles])
    for k in xrange(sample_cycles):
         v_1.append(sample_counts_1[k+i*sample_cycles])
yf = scipy.fftpack.fft(v_1)

# sample spacing
xf = np.linspace(0, 1.0/(2.0*delay*10**-4), (sample_cycles+pid_cycles)*max_repetitions/2)

index_max = [i for i,v in enumerate(yf) if v > 20000]
frq_max = [x * (1.0/(delay*10**-4)) / ((sample_cycles+pid_cycles)*max_repetitions) for x in index_max]
print 'peak frequencies are:'
print frq_max

norm1 = yf / np.linalg.norm(yf)

fig, ax = plt.subplots()
plt.ylim([0,20000])
ax.plot(xf, yf[:len(yf)/2])
plt.title('FFT {0}'.format(folder))
ax.set_xlabel('frequency (Hz)')
ax.set_ylabel('Amplitude (a.u.)')
# ax.set_yticklabels([])
fig.savefig(os.path.join(folder, 'fft.png'))